## Setup and Dependencies

In [38]:
# --- Extension Setup ---
%load_ext autoreload
%load_ext line_profiler
%autoreload 3 -p

# --- Module Imports ---
import sys
sys.path.append("..")  # Adjust if your afml repo is nested differently

# --- Autoreload Target ---
%aimport afml

# --- AFML Initialization ---
# Setup with auto-reload enabled
import afml

# Enhanced setup with all features
components = afml.setup_jupyter(
    enable_mlflow=True,      # Set True if you have mlflow installed
    enable_monitoring=True,    # Cache analytics
)

# --- Environment Diagnostics ---
from pathlib import Path
print(f"Working Dir: {Path.cwd()}")


2025-11-03 23:55:05.597 | INFO     | afml:setup_jupyter:281 - Setting up AFML for Jupyter notebook...
2025-11-03 23:55:05.598 | INFO     | afml.cache:setup_jupyter_cache:583 - Setting up cache for Jupyter notebook...
2025-11-03 23:55:05.599 | DEBUG    | afml.cache:_configure_numba:59 - Numba cache configured: C:\Users\JoeN\AppData\Local\afml\afml\Cache\numba_cache
2025-11-03 23:55:05.603 | INFO     | afml.cache:initialize_cache_system:308 - AFML cache system initialized:
2025-11-03 23:55:05.606 | INFO     | afml.cache:initialize_cache_system:309 -   Joblib cache: C:\Users\JoeN\AppData\Local\afml\afml\Cache\joblib_cache
2025-11-03 23:55:05.609 | INFO     | afml.cache:initialize_cache_system:310 -   Numba cache: C:\Users\JoeN\AppData\Local\afml\afml\Cache\numba_cache
2025-11-03 23:55:05.611 | INFO     | afml.cache:initialize_cache_system:316 -   Loaded stats: 3 functions, 93.8% hit rate
2025-11-03 23:55:05.629 | INFO     | afml.cache.mlflow_integration:__init__:54 - MLflow tracking enabl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
Working Dir: c:\Users\JoeN\Documents\GitHub\Machine-Learning-Blueprint\notebooks


In [ ]:
import time
import warnings
import winsound
from pathlib import Path
from pprint import pprint

import matplotlib.pyplot as plt
import MetaTrader5 as mt5
from sklearn.base import clone
from sklearn.ensemble import (
    AdaBoostClassifier,
    BaggingClassifier,
    RandomForestClassifier,
)
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    log_loss,
    precision_score,
    recall_score,
)
from sklearn.tree import DecisionTreeClassifier

from afml.cross_validation import (
    PurgedKFold,
    PurgedSplit,
    analyze_cross_val_scores,
    ml_cross_val_score,
    probability_weighted_accuracy,
)
from afml.cross_validation.scoring import probability_weighted_accuracy
from afml.data_structures.bars import *
from afml.ensemble.sb_bagging import (
    SequentiallyBootstrappedBaggingClassifier,
    compute_custom_oob_metrics,
    estimate_ensemble_size,
)
from afml.labeling.triple_barrier import (
    add_vertical_barrier,
    get_event_weights,
    triple_barrier_labels,
)
from afml.mt5.load_data import get_bars, get_ticks, login_mt5, save_data_to_parquet
from afml.sample_weights.optimized_attribution import (
    get_weights_by_time_decay_optimized,
)

# from afml.sampling import get_ind_mat_average_uniqueness, get_ind_matrix, seq_bootstrap
from afml.strategies import (
    BollingerStrategy,
    MACrossoverStrategy,
    create_bollinger_features,
    get_entries,
    ma_crossover_feature_engine,
)
from afml.util import get_daily_vol, value_counts_data

warnings.filterwarnings("ignore")
# plt.style.use("seaborn-v0_8-whitegrid")
plt.style.use("dark_background")

In [ ]:
# Add to your startup script or notebook
from afml.cache import get_cache_efficiency_report, print_cache_health

# Check cache health anytime
print_cache_health()

# Find functions with low hit rates or high call counts
df = get_cache_efficiency_report()
df.sort_values('calls', ascending=False).head(10)


CACHE HEALTH REPORT

Overall Statistics:
  Total Functions:     3
  Total Calls:         160
  Overall Hit Rate:    93.8%
  Total Cache Size:    0.00 MB

Top Performers (by hit rate):
  1. triple_barrier_labels: 98.0% (153 calls)
  2. create_bollinger_features: 0.0% (2 calls)
  3. get_event_weights: 0.0% (5 calls)

Worst Performers (by hit rate):
  1. triple_barrier_labels: 98.0% (153 calls)
  2. create_bollinger_features: 0.0% (2 calls)
  3. get_event_weights: 0.0% (5 calls)

Recommendations:
  1. Excellent hit rate (>90%)! Cache system is performing well.




,function,calls,hits,misses,hit_rate,avg_time_ms,cache_size_mb,last_access
1,afml.labeling.triple_barrier.triple_barrier_la...,153,150,3,98.0%,N/A,N/A,N/A
2,afml.labeling.triple_barrier.get_event_weights,5,0,5,0.0%,N/A,N/A,N/A
0,afml.strategies.bollinger_features.create_boll...,2,0,2,0.0%,N/A,N/A,N/A


## 1. Data Preparation

In [ ]:
symbol = "EURUSD"
start_date, end_date = "2018-01-01", "2024-12-31"
sample_start, sample_end = start_date, "2023-12-31"

## 2. Bollinger Band Strategy

In [ ]:
bb_timeframe = "M5"
file = Path(r"..\data\EURUSD_M5_time_2018-01-01-2024-12-31.parq")
bb_time_bars = pd.read_parquet(file)

In [ ]:
bb_period, bb_std = 20, 2 # Bollinger Band parameters
bb_strategy = BollingerStrategy(window=bb_period, num_std=bb_std)
bb_lookback = 10
bb_pt_barrier, bb_sl_barrier, bb_time_horizon = (1, 2, dict(days=1))
min_ret = 5e-5
bb_vol_multiplier = 1

### Time-Bars

In [ ]:
bb_side = bb_strategy.generate_signals(bb_time_bars)
bb_df = bb_time_bars.loc[sample_start : sample_end]

print(f"{bb_strategy.get_strategy_name()} Signals:")
value_counts_data(bb_side.reindex(bb_df.index), verbose=True)

# Volatility target for barriers
vol_lookback = 100
vol_target = get_daily_vol(bb_df.close, vol_lookback) * bb_vol_multiplier
close = bb_df.close
_, t_events = get_entries(bb_strategy, bb_df, filter_threshold=vol_target)

vertical_barriers = add_vertical_barrier(t_events, close, **bb_time_horizon)

Bollinger_w20_std2 Signals:

        count  proportion
side                     
 0    373,536    0.842213
-1     35,095    0.079129
 1     34,886    0.078658



2025-11-03 23:55:11.985 | INFO     | afml.filters.filters:cusum_filter:151 - 19,458 CUSUM-filtered events
2025-11-03 23:55:12.077 | INFO     | afml.strategies.signal_processing:get_entries:105 - Bollinger_w20_std2 | 10,384 (14.84%) trade events selected by CUSUM filter using series.


#### Feature Engineering

In [ ]:
bb_feat = create_bollinger_features(bb_time_bars, bb_period, bb_std)
bb_feat_time = bb_feat.join(bb_side, how="inner")
bb_feat_time.info()
# not_stationary = is_stationary(bb_feat_time)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 516825 entries, 2018-01-02 23:20:00 to 2024-12-31 00:00:00
Data columns (total 49 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   spread               516825 non-null  float32
 1   vol                  516825 non-null  float32
 2   h1_vol               516825 non-null  float32
 3   h4_vol               516825 non-null  float32
 4   d1_vol               516825 non-null  float32
 5   ret                  516825 non-null  float32
 6   ret_5                516825 non-null  float32
 7   ret_10               516825 non-null  float32
 8   ret_1_lag_1          516825 non-null  float32
 9   ret_5_lag_1          516825 non-null  float32
 10  ret_10_lag_1         516825 non-null  float32
 11  ret_1_lag_2          516825 non-null  float32
 12  ret_5_lag_2          516825 non-null  float32
 13  ret_10_lag_2         516825 non-null  float32
 14  ret_1_lag_3          516825 non-nu

#### Triple-Barrier Method

In [ ]:
bb_events_tb = triple_barrier_labels(
    close,
    vol_target,
    t_events,
    pt_sl=[bb_pt_barrier, bb_sl_barrier],
    min_ret=min_ret,
    vertical_barrier_times=vertical_barriers,
    vertical_barrier_zero=True,
    verbose=False,
)

bb_events_tb_time = bb_events_tb.copy()
print(f"Triple-Barrier (pt={bb_pt_barrier}, sl={bb_sl_barrier}, h={bb_time_horizon}):")
value_counts_data(bb_events_tb['bin'], verbose=True)

weights = get_event_weights(bb_events_tb, close)
av_uniqueness = weights['tW'].mean()
print(f"Average Uniqueness: {av_uniqueness:.4f}")

Triple-Barrier (pt=1, sl=2, h={'days': 1}):

     count  proportion
bin                   
-1   5,109    0.505741
 1   4,993    0.494259

Average Uniqueness: 0.7465


In [ ]:
bb_events_tb = triple_barrier_labels(
    close,
    vol_target,
    t_events,
    pt_sl=[bb_pt_barrier, bb_sl_barrier],
    min_ret=min_ret,
    vertical_barrier_times=vertical_barriers,
    side_prediction=bb_side,
    vertical_barrier_zero=True,
    verbose=False,
)

bb_events_tb_time_meta = bb_events_tb.copy()
print(f"Triple-Barrier (pt={bb_pt_barrier}, sl={bb_sl_barrier}, h={bb_time_horizon}):")
value_counts_data(bb_events_tb['bin'], verbose=True)

weights = get_event_weights(bb_events_tb, close)
av_uniqueness = weights['tW'].mean()
print(f"Average Uniqueness: {av_uniqueness:.4f}")

Triple-Barrier (pt=1, sl=2, h={'days': 1}):

     count  proportion
bin                   
1    6,506    0.626601
0    3,877    0.373399

Average Uniqueness: 0.5488


#### Primary Model - CV of Weighting Methods

In [ ]:
from os import cpu_count

# Reserve 1 CPU if you want to do something else during training, otherwise set to -1
N_JOBS = cpu_count() - 1
N_ESTIMATORS = 100
random_state = 7

In [ ]:
cont = bb_events_tb_time.copy()
X = bb_feat_time.reindex(cont.index)
y = cont["bin"]
t1 = cont["t1"]

test_size = 0.2

train, test = PurgedSplit(t1, test_size).split(X)
X_train, X_test, y_train, y_test = (
        X.iloc[train],
        X.iloc[test],
        y.iloc[train],
        y.iloc[test],
    )

cont_train = get_event_weights(cont.iloc[train], bb_df.close)
bb_cont_train = cont_train.copy()

n_splits = 5
pct_embargo = 0.01
cv_gen = PurgedKFold(n_splits, cont_train.t1, pct_embargo)

In [ ]:
avg_u = cont_train.tW.mean()
print(f"Average Uniqueness in Training Set: {avg_u:.4f}")

weighting_schemes = {
    "unweighted": pd.Series(1., index=cont_train.index),
    "uniqueness": cont_train["tW"],
    "return": cont_train["w"],
    }

decay_factors = [0.0, 0.25, 0.5, 0.75]
time_decay_weights = {}
for time_decay in decay_factors:
    decay_w = get_weights_by_time_decay_optimized(
                triple_barrier_events=cont,
                close_index=close.index,
                last_weight=time_decay,
                linear=True,
                av_uniqueness=cont_train["tW"],
            )
    time_decay_weights[f"decay_{time_decay}"] = decay_w
        
# for k, v in time_decay_weights.items():
#     if k.startswith("linear"):
#         weighting_schemes[k] = v

weighting_schemes.keys()

Average Uniqueness in Training Set: 0.7473


dict_keys(['unweighted', 'uniqueness', 'return'])

##### Selection of Best Model

In [ ]:
# Create multiple Random Forest configurations

min_w_leaf = 0.05
max_depth = 6

rf = RandomForestClassifier(
    criterion='entropy',
    n_estimators=N_ESTIMATORS,
    random_state=random_state,
    min_weight_fraction_leaf=min_w_leaf,
    max_depth=max_depth,
    n_jobs=N_JOBS,  # Use all available cores
    )

clf0 = rf
clf1 = clone(rf).set_params(class_weight='balanced_subsample')
clf2 = clone(rf).set_params(max_samples=avg_u)
clf3 = clone(rf).set_params(max_samples=avg_u, class_weight='balanced_subsample')

clfs = {k: v for k, v in zip(['standard', 'balanced_subsample', 'max_samples', 'combined'], [clf0, clf1, clf2, clf3])}
clfs

{'standard': RandomForestClassifier(criterion='entropy', max_depth=6,
                        min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7),
 'balanced_subsample': RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                        max_depth=6, min_weight_fraction_leaf=0.05, n_jobs=3,
                        random_state=7),
 'max_samples': RandomForestClassifier(criterion='entropy', max_depth=6,
                        max_samples=0.7472647467858778,
                        min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7),
 'combined': RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                        max_depth=6, max_samples=0.7472647467858778,
                        min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7)}

In [ ]:
# Find what model produces best CV log loss score

cv_gen = PurgedKFold(n_splits, cont_train.t1, pct_embargo)
cv_scores_d = {k: {} for k in clfs.keys()}
print(rf.__class__.__name__, "Weighting Schemes")
all_clf_scores_df = pd.DataFrame(dtype=pd.StringDtype())
best_models = []
best_score, best_model, best_scheme = None, None, None

for scheme, sample_weights in weighting_schemes.items():
    for param, clf in clfs.items():
        cv_scores = ml_cross_val_score(
            clf, X_train, y_train, cv_gen, 
            sample_weight_train=sample_weights, 
            sample_weight_score=sample_weights,
            scoring="neg_log_loss",
        )
        score = cv_scores.mean()
        cv_scores_d[param][scheme] = score
        best_score = max(best_score, score) if best_score is not None else score
        if score == best_score:
            best_model = param
            best_scheme = scheme
        all_clf_scores_df.loc[param, scheme] = f"{cv_scores.mean():.4f} ± {cv_scores.std():.4f}"

best_clf = clone(clfs[best_model])
print(f"{best_scheme} {best_model} model achieved the best neg_log_loss score of {best_score:.4f}")

print("\nWeighting Scheme CV:")
pprint(all_clf_scores_df)
print(f"\nSelected Best Classifier ({best_model}): {best_clf}")

RandomForestClassifier Weighting Schemes
return standard model achieved the best neg_log_loss score of -0.6613

Weighting Scheme CV:
                          unweighted        uniqueness            return
standard            -0.6937 ± 0.0012  -0.6934 ± 0.0013  -0.6613 ± 0.0057
balanced_subsample  -0.6938 ± 0.0014  -0.6935 ± 0.0015  -0.6614 ± 0.0058
max_samples         -0.6936 ± 0.0009  -0.6934 ± 0.0012  -0.6617 ± 0.0052
combined            -0.6938 ± 0.0008  -0.6935 ± 0.0013  -0.6617 ± 0.0052

Selected Best Classifier (standard): RandomForestClassifier(criterion='entropy', max_depth=6,
                       min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7)


In [ ]:
# Analyze all CV scores for all weighting schemes with the best model

from afml.cross_validation.cross_validation import analyze_cross_val_scores

all_cv_scores_d = {}
all_cms = {}
best_score, best_model = None, None
all_cv_scores_df = pd.DataFrame(dtype=pd.StringDtype())
scoring = 'f1' if set(y_train.unique()) == {0, 1} else 'neg_log_loss'

for scheme, sample_weights in weighting_schemes.items():
    cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        best_clf, X_train, y_train, cv_gen, 
        sample_weight_train=sample_weights, 
        sample_weight_score=sample_weights,
    )
    score = cv_scores[scoring].mean()
    all_cv_scores_d[scheme] = cv_scores
    all_cms[scheme] = cms
    best_score = max(best_score, score) if best_score is not None else score
    if score == best_score:
        best_scheme = scheme
    for idx, row in cv_scores_df.iterrows():
        all_cv_scores_df.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"

print("Weighting Scheme CV:")
pprint(all_cv_scores_df.T)
print(f"\n{best_scheme} model achieved the best {scoring} score of {best_score:.4f}\n")

Weighting Scheme CV:
                   accuracy              pwa      neg_log_loss  \
unweighted  0.5047 ± 0.0161  0.5085 ± 0.0171  -0.6937 ± 0.0012   
uniqueness  0.5094 ± 0.0184  0.5133 ± 0.0147  -0.6934 ± 0.0013   
return      0.6249 ± 0.0146  0.6343 ± 0.0139  -0.6613 ± 0.0057   

                  precision           recall               f1  
unweighted  0.4963 ± 0.0307  0.3294 ± 0.0785  0.3893 ± 0.0475  
uniqueness  0.5027 ± 0.0407  0.3409 ± 0.0925  0.3972 ± 0.0626  
return      0.6183 ± 0.0177  0.6025 ± 0.0327  0.6101 ± 0.0247  

return model achieved the best neg_log_loss score of -0.6613



Test if time-decay improves performance of best model

In [ ]:
best_model_decay_cv_scores = all_cv_scores_df[[best_scheme]]

for scheme, decay_factor in time_decay_weights.items():
    sample_weights = weighting_schemes[best_scheme] * decay_factor
    cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        best_clf, X_train, y_train, cv_gen, 
        sample_weight_train=sample_weights, 
        sample_weight_score=sample_weights,
    )
    score = cv_scores[scoring].mean()
    scheme = f"{best_scheme}_{scheme}"
    all_cv_scores_d[scheme] = cv_scores
    all_cms[scheme] = cms
    best_score = max(best_score, score) if best_score is not None else score
    for idx, row in cv_scores_df.iterrows():
        best_model_decay_cv_scores.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"
    if score == best_score:
        best_scheme = scheme
        weighting_schemes[best_scheme] = sample_weights
        all_cv_scores_df[scheme] = best_model_decay_cv_scores[scheme]
        

print(f"\nBest Weighting Scheme CV - {best_scheme.title()}:")
pprint(best_model_decay_cv_scores)

print(f"\n{best_scheme} model achieved the best {scoring} score of {best_score:.4f}\n")


Best Weighting Scheme CV - Return:
                        return  return_decay_0.0 return_decay_0.25  \
accuracy       0.6249 ± 0.0146   0.6236 ± 0.0117   0.6236 ± 0.0120   
pwa            0.6343 ± 0.0139   0.6332 ± 0.0144   0.6331 ± 0.0138   
neg_log_loss  -0.6613 ± 0.0057  -0.6628 ± 0.0057  -0.6623 ± 0.0056   
precision      0.6183 ± 0.0177   0.6184 ± 0.0162   0.6179 ± 0.0169   
recall         0.6025 ± 0.0327   0.5966 ± 0.0310   0.5983 ± 0.0317   
f1             0.6101 ± 0.0247   0.6071 ± 0.0227   0.6077 ± 0.0232   

              return_decay_0.5 return_decay_0.75  
accuracy       0.6224 ± 0.0121   0.6225 ± 0.0152  
pwa            0.6336 ± 0.0137   0.6328 ± 0.0143  
neg_log_loss  -0.6618 ± 0.0055  -0.6621 ± 0.0058  
precision      0.6159 ± 0.0176   0.6156 ± 0.0191  
recall         0.5999 ± 0.0297   0.6007 ± 0.0335  
f1             0.6076 ± 0.0225   0.6079 ± 0.0257  

return model achieved the best neg_log_loss score of -0.6613



In [ ]:
all_cms[scheme] = cms
# pprint(all_cms, sort_dicts=False)
pprint(all_cms[best_scheme], sort_dicts=False)

[{'fold': 1, 'TN': 597.74, 'FP': 306.17, 'FN': 329.18, 'TP': 548.78},
 {'fold': 2, 'TN': 383.48, 'FP': 200.75, 'FN': 226.25, 'TP': 305.54},
 {'fold': 3, 'TN': 536.6, 'FP': 322.27, 'FN': 310.15, 'TP': 566.3},
 {'fold': 4, 'TN': 424.97, 'FP': 234.74, 'FN': 280.48, 'TP': 352.45},
 {'fold': 5, 'TN': 722.81, 'FP': 404.05, 'FN': 401.37, 'TP': 627.92}]


##### Sequential Bootstrap

In [61]:
# Base estimator for use with sequential bootstrapping
# I chose it beacause the default behaviour of RF is to set max_features='sqrt'
base_rf = clone(best_clf)
base_rf.set_params(bootstrap=False, n_estimators=1, random_state=None, n_jobs=1)

seq_rf = SequentiallyBootstrappedBaggingClassifier(
    samples_info_sets=cont_train.t1,
    price_bars_index=bb_df.index,
    estimator=base_rf,
    n_estimators=20, # set low to save time
    max_features=1,
    max_samples=0.5,
    bootstrap_features=True,
    oob_score=True,
    n_jobs=N_JOBS,
    random_state=random_state,
    verbose=False,
)

for scheme in ("unweighted", best_scheme):
    w = weighting_schemes[best_scheme] if scheme is not "unweighted" else None 
    cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
            seq_rf, X_train, y_train, cv_gen, 
            sample_weight_train=w, 
            sample_weight_score=w,
        )
    scheme = f'seq_bootstrap_{scheme}'
    all_cms[scheme] = cms
    
    for idx, row in cv_scores_df.iterrows():
        all_cv_scores_df.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"

bb_all_cv_scores_df_primary = all_cv_scores_df.copy()
all_cv_scores_df

Reloading 'afml.cross_validation.cross_validation'.


,unweighted,uniqueness,return,seq_bootstrap_unweighted,seq_bootstrap_return
accuracy,0.5047 ± 0.0161,0.5094 ± 0.0184,0.6249 ± 0.0146,0.4983 ± 0.0076,0.5862 ± 0.0117
pwa,0.5085 ± 0.0171,0.5133 ± 0.0147,0.6343 ± 0.0139,0.5054 ± 0.0079,0.6014 ± 0.0118
neg_log_loss,-0.6937 ± 0.0012,-0.6934 ± 0.0013,-0.6613 ± 0.0057,-0.6935 ± 0.0005,-0.6823 ± 0.0025
precision,0.4963 ± 0.0307,0.5027 ± 0.0407,0.6183 ± 0.0177,0.4714 ± 0.0202,0.5906 ± 0.0193
recall,0.3294 ± 0.0785,0.3409 ± 0.0925,0.6025 ± 0.0327,0.2219 ± 0.0500,0.4965 ± 0.0592
f1,0.3893 ± 0.0475,0.3972 ± 0.0626,0.6101 ± 0.0247,0.2989 ± 0.0504,0.5375 ± 0.0381


In [ ]:
w = weighting_schemes[best_scheme]
rf = best_clf.set_params(oob_score=True).fit(
    X_train, y_train, sample_weight=w,
)

time0 = time.time()
seq_rf.set_params(oob_score=True, n_estimators=50).fit(
    X_train, y_train, sample_weight=w,
)
seq_rfu = clone(seq_rf).fit(X_train, y_train)
time1 = pd.Timedelta(seconds=time.time() - time0).round('1s')
print(f"Sequential Bootstrap done in {time1}")

ensembles = {
    "standard_rf": {"classifier": rf, 
                    "pred": rf.predict(X_test),
                    "prob": rf.predict_proba(X_test),
                    "oob": rf.oob_score_,
                },
    "sequential_rf": {"classifier": seq_rf, 
                      "pred": seq_rf.predict(X_test),
                      "prob": seq_rf.predict_proba(X_test),
                      "oob": seq_rf.oob_score_,
                      },
    "sequential_rf_unweighted": {"classifier": seq_rfu, 
                                 "pred": seq_rfu.predict(X_test),
                                 "prob": seq_rfu.predict_proba(X_test),
                                 "oob": seq_rfu.oob_score_,
                                 },
}

scoring_methods = {
            "accuracy": accuracy_score,
            "pwa": probability_weighted_accuracy,
            "neg_log_loss": log_loss,
            "precision": precision_score,
            "recall": recall_score,
            "f1": f1_score,
        }

all_scores_oos = pd.DataFrame()

for clf in ensembles.keys():
    for method, scoring in scoring_methods.items():
        if scoring in (probability_weighted_accuracy, log_loss):
            y_pred = ensembles[clf]["prob"]
        else:
            y_pred = ensembles[clf]["pred"]
        score = scoring(y_test, y_pred)
        if method == "neg_log_loss":
            score *= -1
        all_scores_oos.loc[method, clf] = score

bb_all_scores_oos_primary = all_scores_oos.copy()
all_scores_oos.round(4)

Sequential Bootstrap done in 0 days 00:00:02


,standard_rf,sequential_rf
accuracy,0.4812,0.5134
pwa,0.4865,0.5134
neg_log_loss,-0.7218,-17.5401
precision,0.4946,0.5134
recall,0.4860,1.0000
f1,0.4903,0.6784


In [ ]:
winsound.Beep(1000, 1000)

#### Meta-Labelled CV of Weighting Methods

In [ ]:
from os import cpu_count

# Reserve 1 CPU if you want to do something else during training, otherwise set to -1
N_JOBS = cpu_count() - 1
N_ESTIMATORS = 100
random_state = 7

In [ ]:
cont = bb_events_tb_time_meta.copy()
X = bb_feat_time.reindex(cont.index)
y = cont["bin"]
t1 = cont["t1"]

test_size = 0.2

train, test = PurgedSplit(t1, test_size).split(X)
X_train, X_test, y_train, y_test = (
        X.iloc[train],
        X.iloc[test],
        y.iloc[train],
        y.iloc[test],
    )

cont_train = get_event_weights(cont.iloc[train], bb_df.close)
bb_cont_train = cont_train.copy()

n_splits = 5
pct_embargo = 0.01
cv_gen = PurgedKFold(n_splits, cont_train.t1, pct_embargo)

In [ ]:
avg_u = cont_train.tW.mean()
print(f"Average Uniqueness in Training Set: {avg_u:.4f}")

weighting_schemes = {
    "unweighted": pd.Series(1., index=cont_train.index),
    "uniqueness": cont_train["tW"],
    "return": cont_train["w"],
    }

decay_factors = [0.0, 0.25, 0.5, 0.75]
time_decay_weights = {}
for time_decay in decay_factors:
    decay_w = get_weights_by_time_decay_optimized(
                triple_barrier_events=cont,
                close_index=close.index,
                last_weight=time_decay,
                linear=True,
                av_uniqueness=cont_train["tW"],
            )
    time_decay_weights[f"decay_{time_decay}"] = decay_w
        
# for k, v in time_decay_weights.items():
#     if k.startswith("linear"):
#         weighting_schemes[k] = v

weighting_schemes.keys()

Average Uniqueness in Training Set: 0.5473


dict_keys(['unweighted', 'uniqueness', 'return'])

##### Selection of Best Model

In [ ]:
# Create multiple Random Forest configurations

min_w_leaf = 0.05
max_depth = 6

rf = RandomForestClassifier(
    criterion='entropy',
    n_estimators=N_ESTIMATORS,
    random_state=random_state,
    min_weight_fraction_leaf=min_w_leaf,
    max_depth=max_depth,
    n_jobs=N_JOBS,  # Use all available cores
    )

clf0 = rf
clf1 = clone(rf).set_params(class_weight='balanced_subsample')
clf2 = clone(rf).set_params(max_samples=avg_u)
clf3 = clone(rf).set_params(max_samples=avg_u, class_weight='balanced_subsample')

clfs = {k: v for k, v in zip(['standard', 'balanced_subsample', 'max_samples', 'combined'], [clf0, clf1, clf2, clf3])}
clfs

{'standard': RandomForestClassifier(criterion='entropy', max_depth=6,
                        min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7),
 'balanced_subsample': RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                        max_depth=6, min_weight_fraction_leaf=0.05, n_jobs=3,
                        random_state=7),
 'max_samples': RandomForestClassifier(criterion='entropy', max_depth=6,
                        max_samples=0.5473256253464687,
                        min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7),
 'combined': RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                        max_depth=6, max_samples=0.5473256253464687,
                        min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7)}

In [ ]:
# Find what model produces best CV log loss score

cv_gen = PurgedKFold(n_splits, cont_train.t1, pct_embargo)
cv_scores_d = {k: {} for k in clfs.keys()}
print(rf.__class__.__name__, "Weighting Schemes")
all_clf_scores_df = pd.DataFrame(dtype=pd.StringDtype())
best_models = []
best_score, best_model, best_scheme = None, None, None

for scheme, sample_weights in weighting_schemes.items():
    for param, clf in clfs.items():
        cv_scores = ml_cross_val_score(
            clf, X_train, y_train, cv_gen, 
            sample_weight_train=sample_weights, 
            sample_weight_score=sample_weights,
            scoring="neg_log_loss",
        )
        score = cv_scores.mean()
        cv_scores_d[param][scheme] = score
        best_score = max(best_score, score) if best_score is not None else score
        if score == best_score:
            best_model = param
            best_scheme = scheme
        all_clf_scores_df.loc[param, scheme] = f"{cv_scores.mean():.4f} ± {cv_scores.std():.4f}"

best_clf = clone(clfs[best_model])
print(f"{best_scheme} {best_model} model achieved the best neg_log_loss score of {best_score:.4f}")

print("\nWeighting Scheme CV:")
pprint(all_clf_scores_df)
print(f"\nSelected Best Classifier ({best_model}): {best_clf}")

RandomForestClassifier Weighting Schemes
return max_samples model achieved the best neg_log_loss score of -0.6597

Weighting Scheme CV:
                          unweighted        uniqueness            return
standard            -0.6611 ± 0.0095  -0.6647 ± 0.0099  -0.6600 ± 0.0023
balanced_subsample  -0.6919 ± 0.0034  -0.6956 ± 0.0045  -0.6847 ± 0.0036
max_samples         -0.6612 ± 0.0101  -0.6646 ± 0.0099  -0.6597 ± 0.0023
combined            -0.6919 ± 0.0029  -0.6954 ± 0.0048  -0.6831 ± 0.0042

Selected Best Classifier (max_samples): RandomForestClassifier(criterion='entropy', max_depth=6,
                       max_samples=0.5473256253464687,
                       min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7)


In [ ]:
# Analyze all CV scores for all weighting schemes with the best model

from afml.cross_validation.cross_validation import analyze_cross_val_scores

all_cv_scores_d = {}
all_cms = {}
best_score, best_model = None, None
all_cv_scores_df = pd.DataFrame(dtype=pd.StringDtype())
scoring = 'f1' if set(y_train.unique()) == {0, 1} else 'neg_log_loss'

for scheme, sample_weights in weighting_schemes.items():
    cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        best_clf, X_train, y_train, cv_gen, 
        sample_weight_train=sample_weights, 
        sample_weight_score=sample_weights,
    )
    score = cv_scores[scoring].mean()
    all_cv_scores_d[scheme] = cv_scores
    all_cms[scheme] = cms
    best_score = max(best_score, score) if best_score is not None else score
    if score == best_score:
        best_scheme = scheme
    for idx, row in cv_scores_df.iterrows():
        all_cv_scores_df.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"

print("Weighting Scheme CV:")
pprint(all_cv_scores_df.T)
print(f"\n{best_scheme} model achieved the best {scoring} score of {best_score:.4f}\n")

Weighting Scheme CV:
                   accuracy              pwa      neg_log_loss  \
unweighted  0.6262 ± 0.0159  0.6310 ± 0.0181  -0.6612 ± 0.0101   
uniqueness  0.6172 ± 0.0166  0.6258 ± 0.0185  -0.6646 ± 0.0099   
return      0.6245 ± 0.0048  0.6347 ± 0.0052  -0.6597 ± 0.0023   

                  precision           recall               f1  
unweighted  0.6262 ± 0.0159  1.0000 ± 0.0000  0.7700 ± 0.0119  
uniqueness  0.6175 ± 0.0164  0.9989 ± 0.0013  0.7631 ± 0.0126  
return      0.1482 ± 0.2964  0.0013 ± 0.0027  0.0026 ± 0.0053  

unweighted model achieved the best f1 score of 0.7700



Test if time-decay improves performance of best model

In [ ]:
best_model_decay_cv_scores = all_cv_scores_df[[best_scheme]]

for scheme, decay_factor in time_decay_weights.items():
    sample_weights = weighting_schemes[best_scheme] * decay_factor
    cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        best_clf, X_train, y_train, cv_gen, 
        sample_weight_train=sample_weights, 
        sample_weight_score=sample_weights,
    )
    score = cv_scores[scoring].mean()
    scheme = f"{best_scheme}_{scheme}"
    all_cv_scores_d[scheme] = cv_scores
    all_cms[scheme] = cms
    best_score = max(best_score, score) if best_score is not None else score
    for idx, row in cv_scores_df.iterrows():
        best_model_decay_cv_scores.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"
    if score == best_score:
        best_scheme = scheme
        weighting_schemes[best_scheme] = sample_weights
        all_cv_scores_df[scheme] = best_model_decay_cv_scores[scheme]
        

print(f"\nBest Weighting Scheme CV - {best_scheme.title()}:")
pprint(best_model_decay_cv_scores)

print(f"\n{best_scheme} model achieved the best {scoring} score of {best_score:.4f}\n")


Best Weighting Scheme CV - Unweighted:
                    unweighted unweighted_decay_0.0 unweighted_decay_0.25  \
accuracy       0.6262 ± 0.0159      0.6253 ± 0.0152       0.6258 ± 0.0155   
pwa            0.6310 ± 0.0181      0.6283 ± 0.0153       0.6300 ± 0.0163   
neg_log_loss  -0.6612 ± 0.0101     -0.6622 ± 0.0086      -0.6614 ± 0.0091   
precision      0.6262 ± 0.0159      0.6253 ± 0.0152       0.6258 ± 0.0155   
recall         1.0000 ± 0.0000      1.0000 ± 0.0000       1.0000 ± 0.0000   
f1             0.7700 ± 0.0119      0.7693 ± 0.0114       0.7697 ± 0.0116   

             unweighted_decay_0.5 unweighted_decay_0.75  
accuracy          0.6260 ± 0.0157       0.6261 ± 0.0158  
pwa               0.6303 ± 0.0172       0.6306 ± 0.0177  
neg_log_loss     -0.6613 ± 0.0096      -0.6613 ± 0.0099  
precision         0.6260 ± 0.0157       0.6261 ± 0.0158  
recall            1.0000 ± 0.0000       1.0000 ± 0.0000  
f1                0.7699 ± 0.0117       0.7699 ± 0.0118  

unweighted mo

In [ ]:
all_cms[scheme] = cms
# pprint(all_cms, sort_dicts=False)
pprint(all_cms[best_scheme], sort_dicts=False)

KeyError: None

##### Sequential Bootstrap

In [ ]:
# Base estimator for use with sequential bootstrapping
# I chose it beacause the default behaviour of RF is to set max_features='sqrt'
base_rf = clone(best_clf).set_params(bootstrap=False, n_estimators=1, random_state=None, n_jobs=1)

seq_rf = SequentiallyBootstrappedBaggingClassifier(
    samples_info_sets=cont_train.t1,
    price_bars_index=bb_df.index,
    estimator=base_rf,
    n_estimators=20, # set low to save time
    max_features=1,
    max_samples=1,
    bootstrap_features=True,
    oob_score=True,
    n_jobs=N_JOBS,
    random_state=random_state,
    verbose=False,
)

w = weighting_schemes[best_scheme]
cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        seq_rf, X_train, y_train, cv_gen, 
        sample_weight_train=w, 
        sample_weight_score=w,
    )
all_cms[scheme] = cms

scheme = 'seq_bootstrap'
for idx, row in cv_scores_df.iterrows():
    all_cv_scores_df.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"

bb_all_cv_scores_df_meta = all_cv_scores_df.copy()
all_cv_scores_df

SequentiallyBootstrappedBaggingClassifier(estimator=RandomForestClassifier(bootstrap=False,
                                                                           criterion='entropy',
                                                                           max_depth=6,
                                                                           min_weight_fraction_leaf=0.05,
                                                                           n_estimators=1,
                                                                           n_jobs=3),
                                          max_features=1, n_estimators=100,
                                          n_jobs=3, oob_score=True,
                                          price_bars_index=DatetimeIndex(['2018-01-01 23:05:00', '2018-01-01 23:10:00',
               '2018-01-01 23:15:00', '2018-01-01 23:20:00',...
2018-01-03 01:30:00   2018-01-03 01:50:00
2018-01-03 02:40:00   2018-01-03 04:00:00
2018-01-03 05:35:00   2018-01

KeyboardInterrupt: 

In [ ]:
w = weighting_schemes[best_scheme]
rf = best_clf.set_params(oob_score=True).fit(
    X_train, y_train, sample_weight=w,
)

time0 = time.time()
seq_rf.set_params(oob_score=True).fit(
    X_train, y_train, sample_weight=w,
)
time1 = pd.Timedelta(seconds=time.time() - time0).round('1s')
print(f"Sequential Bootstrap done in {time1}")

ensembles = {
    "standard_rf": {"classifier": rf, 
                    "pred": rf.predict(X_test),
                    "prob": rf.predict_proba(X_test),
                    "oob": rf.oob_score_,
                },
    "sequential_rf": {"classifier": seq_rf, 
                      "pred": seq_rf.predict(X_test),
                      "prob": seq_rf.predict_proba(X_test),
                      "oob": seq_rf.oob_score_,
                      },
}

scoring_methods = {
            "accuracy": accuracy_score,
            "pwa": probability_weighted_accuracy,
            "neg_log_loss": log_loss,
            "precision": precision_score,
            "recall": recall_score,
            "f1": f1_score,
        }

all_scores_oos = pd.DataFrame()

for clf in ensembles.keys():
    for method, scoring in scoring_methods.items():
        if scoring in (probability_weighted_accuracy, log_loss):
            y_pred = ensembles[clf]["prob"]
        else:
            y_pred = ensembles[clf]["pred"]
        score = scoring(y_test, y_pred)
        if method == "neg_log_loss":
            score *= -1
        all_scores_oos.loc[method, clf] = score

bb_all_scores_oos_meta = all_scores_oos.copy()
all_scores_oos.round(4)

Sequential Bootstrap done in 0 days 00:12:06


,standard_rf,sequential_rf,sequential_rf_avgu,sequential_rf_unweighted_avgu
accuracy,0.4855,0.5040,0.5013,0.5000
pwa,0.4918,0.5024,0.4899,0.4916
neg_log_loss,-0.7232,-0.7016,-0.7036,-0.7033
precision,0.4888,0.5077,0.5054,0.5038
recall,0.4852,0.4728,0.4315,0.4315
f1,0.4870,0.4896,0.4655,0.4649


In [ ]:
winsound.Beep(1000, 1000)

## 3. Moving Average Crossover Strategy

In [ ]:
from afml.strategies.ma_crossover_feature_engine import ForexFeatureEngine

ma_timeframe = "M5"
file = Path(r"..\data\EURUSD_M15_time_2018-01-01-2024-12-31.parq")
ma_time_bars = pd.read_parquet(file)

fast_window, slow_window = 20, 50
ma_strategy = MACrossoverStrategy(fast_window, slow_window)
ma_pt_barrier, ma_sl_barrier, ma_time_horizon = (0, 2, dict(days=5))
ma_vol_multiplier = 1

### Time-Bars

In [ ]:
ma_side = ma_strategy.generate_signals(ma_time_bars)
ma_df = ma_time_bars.loc[sample_start : sample_end]


print(f"{ma_strategy.get_strategy_name()} Signals:")
value_counts_data(ma_side.reindex(ma_df.index), verbose=True)

# Volatility target for barriers
vol_lookback = fast_window
vol_target = get_daily_vol(ma_df.close, vol_lookback) * ma_vol_multiplier
close = ma_df.close

thres = vol_target.mean()
_, t_events = get_entries(ma_strategy, ma_df, filter_threshold=vol_target.mean())

vertical_barriers = add_vertical_barrier(t_events, close, **ma_time_horizon)
linear_decay = False

2025-10-27 06:58:41.959 | INFO     | afml.filters.filters:cusum_filter:151 - 12,748 CUSUM-filtered events


MACrossover_20_50 Signals:

       count  proportion
side                    
-1    61,845    0.502062
 1    61,287    0.497532
 0        50    0.000406



2025-10-27 06:58:42.059 | INFO     | afml.strategies.signal_processing:get_entries:105 - MACrossover_20_50 | 12,744 (10.35%) trade events selected by CUSUM filter (threshold = 0.1252%).


#### Feature Engineering

In [ ]:
ma_feat_engine = ForexFeatureEngine(pair_name=symbol)
ma_feat_time = ma_feat_engine.calculate_all_features(ma_time_bars, ma_timeframe, lr_period=(5, 20))
ma_feat_time.info()

Memory usage reduced from 106.62 MB to 55.49 MB (48.0% reduction)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 172386 entries, 2018-01-01 23:15:00 to 2024-12-31 00:00:00
Data columns (total 94 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ma_10                           172386 non-null  float32
 1   ma_20                           172386 non-null  float32
 2   ma_50                           172386 non-null  float32
 3   ma_100                          172386 non-null  float32
 4   ma_200                          172386 non-null  float32
 5   ma_10_20_cross                  172386 non-null  float64
 6   ma_20_50_cross                  172386 non-null  float64
 7   ma_50_200_cross                 172386 non-null  float64
 8   ma_spread_10_20                 172386 non-null  float32
 9   ma_spread_20_50                 172386 non-null  float32
 10  ma_spread_50_200                172386 n

In [ ]:
for i, col in enumerate(ma_feat_time):
    print(f"{i:>3}. {col}")

  0. ma_10
  1. ma_20
  2. ma_50
  3. ma_100
  4. ma_200
  5. ma_10_20_cross
  6. ma_20_50_cross
  7. ma_50_200_cross
  8. ma_spread_10_20
  9. ma_spread_20_50
 10. ma_spread_50_200
 11. ma_20_slope
 12. ma_50_slope
 13. price_above_ma_20
 14. price_above_ma_50
 15. ma_ribbon_aligned
 16. atr_14
 17. atr_21
 18. atr_regime
 19. realized_vol_10
 20. realized_vol_20
 21. realized_vol_50
 22. vol_of_vol
 23. hl_range
 24. hl_range_ma
 25. hl_range_regime
 26. bb_upper
 27. bb_lower
 28. bb_percent
 29. bb_bandwidth
 30. bb_squeeze
 31. efficiency_ratio_14
 32. efficiency_ratio_30
 33. adx_14
 34. dmp_14
 35. dmn_14
 36. adx_trend_strength
 37. adx_trend_direction
 38. trend_window
 39. trend_slope
 40. trend_t_value
 41. trend_rsquared
 42. trend_ret
 43. roc_10
 44. roc_20
 45. momentum_14
 46. hh_ll_20
 47. trend_persistence
 48. return_skew_20
 49. return_kurtosis_20
 50. var_95
 51. cvar_95
 52. market_stress
 53. current_drawdown
 54. days_since_high
 55. hour_sin_h1
 56. hour_cos_h1

#### Triple-Barrier Method

In [ ]:
ma_events_tb = triple_barrier_labels(
    close=close,
    target=vol_target,
    t_events=t_events,
    pt_sl=[ma_pt_barrier, ma_sl_barrier],
    min_ret=min_ret,
    vertical_barrier_times=vertical_barriers,
    side_prediction=ma_side,
    vertical_barrier_zero=False,
    verbose=False,
)
ma_events_tb_time = ma_events_tb.copy()
ma_events_tb.info()

print(f"Triple-Barrier (pt={ma_pt_barrier}, sl={ma_sl_barrier}, h={ma_time_horizon}):")
value_counts_data(ma_events_tb.bin, verbose=True)

weights = get_event_weights(ma_events_tb, close)
av_uniqueness = weights['tW'].mean()
print(f"Average Uniqueness: {av_uniqueness:.4f}")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12716 entries, 2018-01-03 02:45:00 to 2022-12-30 12:30:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   t1      12716 non-null  datetime64[ns]
 1   trgt    12716 non-null  float64       
 2   ret     12716 non-null  float64       
 3   bin     12716 non-null  int8          
 4   side    12716 non-null  int8          
dtypes: datetime64[ns](1), float64(2), int8(2)
memory usage: 422.2 KB
Triple-Barrier (pt=0, sl=2, h={'days': 5}):

     count  proportion
bin                   
0    9,109    0.716342
1    3,607    0.283658

Average Uniqueness: 0.0668


#### Meta Model - CV of Weighting Methods

In [ ]:
from os import cpu_count

# Reserve 1 CPU if you want to do something else during training, otherwise set to -1
N_JOBS = cpu_count() - 1
N_ESTIMATORS = 100
random_state = 7

In [ ]:
cont = ma_events_tb_time.copy()
X = ma_feat_time.reindex(cont.index)
y = cont["bin"]
t1 = cont["t1"]

test_size = 0.2

train, test = PurgedSplit(t1, test_size).split(X)
X_train, X_test, y_train, y_test = (
        X.iloc[train],
        X.iloc[test],
        y.iloc[train],
        y.iloc[test],
    )

cont_train = get_event_weights(cont.iloc[train], bb_df.close)
bb_cont_train = cont_train.copy()

n_splits = 5
pct_embargo = 0.01
cv_gen = PurgedKFold(n_splits, cont_train.t1, pct_embargo)

In [ ]:
avg_u = cont_train.tW.mean()
print(f"Average Uniqueness in Training Set: {avg_u:.4f}")

weighting_schemes = {
    "unweighted": pd.Series(1., index=cont_train.index),
    "uniqueness": cont_train["tW"],
    "return": cont_train["w"],
    }

decay_factors = [0.0, 0.25, 0.5, 0.75]
time_decay_weights = {}
for time_decay in decay_factors:
    decay_w = get_weights_by_time_decay_optimized(
                triple_barrier_events=cont,
                close_index=close.index,
                last_weight=time_decay,
                linear=True,
                av_uniqueness=cont_train["tW"],
            )
    time_decay_weights[f"decay_{time_decay}"] = decay_w
        
# for k, v in time_decay_weights.items():
#     if k.startswith("linear"):
#         weighting_schemes[k] = v

weighting_schemes.keys()

Average Uniqueness in Training Set: 0.7473


dict_keys(['unweighted', 'uniqueness', 'return'])

##### Selection of Best Model

In [ ]:
# Create multiple Random Forest configurations

min_w_leaf = 0.05
max_depth = 6

rf = RandomForestClassifier(
    criterion='entropy',
    n_estimators=N_ESTIMATORS,
    random_state=random_state,
    min_weight_fraction_leaf=min_w_leaf,
    max_depth=max_depth,
    n_jobs=N_JOBS,  # Use all available cores
    )

clf0 = rf
clf1 = clone(rf).set_params(class_weight='balanced_subsample')
clf2 = clone(rf).set_params(max_samples=avg_u)
clf3 = clone(rf).set_params(max_samples=avg_u, class_weight='balanced_subsample')

clfs = {k: v for k, v in zip(['standard', 'balanced_subsample', 'max_samples', 'combined'], [clf0, clf1, clf2, clf3])}
clfs

{'standard': RandomForestClassifier(criterion='entropy', max_depth=6,
                        min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7),
 'balanced_subsample': RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                        max_depth=6, min_weight_fraction_leaf=0.05, n_jobs=3,
                        random_state=7),
 'max_samples': RandomForestClassifier(criterion='entropy', max_depth=6,
                        max_samples=0.7472647467858778,
                        min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7),
 'combined': RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                        max_depth=6, max_samples=0.7472647467858778,
                        min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7)}

In [ ]:
# Find what model produces best CV log loss score

cv_gen = PurgedKFold(n_splits, cont_train.t1, pct_embargo)
cv_scores_d = {k: {} for k in clfs.keys()}
print(rf.__class__.__name__, "Weighting Schemes")
all_clf_scores_df = pd.DataFrame(dtype=pd.StringDtype())
best_models = []
best_score, best_model, best_scheme = None, None, None

for scheme, sample_weights in weighting_schemes.items():
    for param, clf in clfs.items():
        cv_scores = ml_cross_val_score(
            clf, X_train, y_train, cv_gen, 
            sample_weight_train=sample_weights, 
            sample_weight_score=sample_weights,
            scoring="neg_log_loss",
        )
        score = cv_scores.mean()
        cv_scores_d[param][scheme] = score
        best_score = max(best_score, score) if best_score is not None else score
        if score == best_score:
            best_model = param
            best_scheme = scheme
        all_clf_scores_df.loc[param, scheme] = f"{cv_scores.mean():.4f} ± {cv_scores.std():.4f}"

best_clf = clone(clfs[best_model])
print(f"{best_scheme} {best_model} model achieved the best neg_log_loss score of {best_score:.4f}")

print("\nWeighting Scheme CV:")
pprint(all_clf_scores_df)
print(f"\nSelected Best Classifier ({best_model}): {best_clf}")

RandomForestClassifier Weighting Schemes
return standard model achieved the best neg_log_loss score of -0.6613

Weighting Scheme CV:
                          unweighted        uniqueness            return
standard            -0.6937 ± 0.0012  -0.6934 ± 0.0013  -0.6613 ± 0.0057
balanced_subsample  -0.6938 ± 0.0014  -0.6935 ± 0.0015  -0.6614 ± 0.0058
max_samples         -0.6936 ± 0.0009  -0.6934 ± 0.0012  -0.6617 ± 0.0052
combined            -0.6938 ± 0.0008  -0.6935 ± 0.0013  -0.6617 ± 0.0052

Selected Best Classifier (standard): RandomForestClassifier(criterion='entropy', max_depth=6,
                       min_weight_fraction_leaf=0.05, n_jobs=3, random_state=7)


In [ ]:
# Analyze all CV scores for all weighting schemes with the best model

from afml.cross_validation.cross_validation import analyze_cross_val_scores

all_cv_scores_d = {}
all_cms = {}
best_score, best_model = None, None
all_cv_scores_df = pd.DataFrame(dtype=pd.StringDtype())
scoring = 'f1' if set(y_train.unique()) == {0, 1} else 'neg_log_loss'

for scheme, sample_weights in weighting_schemes.items():
    cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        best_clf, X_train, y_train, cv_gen, 
        sample_weight_train=sample_weights, 
        sample_weight_score=sample_weights,
    )
    score = cv_scores[scoring].mean()
    all_cv_scores_d[scheme] = cv_scores
    all_cms[scheme] = cms
    best_score = max(best_score, score) if best_score is not None else score
    if score == best_score:
        best_scheme = scheme
    for idx, row in cv_scores_df.iterrows():
        all_cv_scores_df.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"

print("Weighting Scheme CV:")
pprint(all_cv_scores_df.T)
print(f"\n{best_scheme} model achieved the best {scoring} score of {best_score:.4f}\n")

Weighting Scheme CV:
                   accuracy              pwa      neg_log_loss  \
unweighted  0.5047 ± 0.0161  0.5085 ± 0.0171  -0.6937 ± 0.0012   
uniqueness  0.5094 ± 0.0184  0.5133 ± 0.0147  -0.6934 ± 0.0013   
return      0.6249 ± 0.0146  0.6343 ± 0.0139  -0.6613 ± 0.0057   

                  precision           recall               f1  
unweighted  0.4963 ± 0.0307  0.3294 ± 0.0785  0.3893 ± 0.0475  
uniqueness  0.5027 ± 0.0407  0.3409 ± 0.0925  0.3972 ± 0.0626  
return      0.6183 ± 0.0177  0.6025 ± 0.0327  0.6101 ± 0.0247  

return model achieved the best neg_log_loss score of -0.6613



Test if time-decay improves performance of best model

In [ ]:
best_model_decay_cv_scores = all_cv_scores_df[[best_scheme]]

for scheme, decay_factor in time_decay_weights.items():
    sample_weights = weighting_schemes[best_scheme] * decay_factor
    cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        best_clf, X_train, y_train, cv_gen, 
        sample_weight_train=sample_weights, 
        sample_weight_score=sample_weights,
    )
    score = cv_scores[scoring].mean()
    scheme = f"{best_scheme}_{scheme}"
    all_cv_scores_d[scheme] = cv_scores
    all_cms[scheme] = cms
    best_score = max(best_score, score) if best_score is not None else score
    for idx, row in cv_scores_df.iterrows():
        best_model_decay_cv_scores.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"
    if score == best_score:
        best_scheme = scheme
        weighting_schemes[best_scheme] = sample_weights
        all_cv_scores_df[scheme] = best_model_decay_cv_scores[scheme]
        

print(f"\nBest Weighting Scheme CV - {best_scheme.title()}:")
pprint(best_model_decay_cv_scores)

print(f"\n{best_scheme} model achieved the best {scoring} score of {best_score:.4f}\n")


Best Weighting Scheme CV - Return:
                        return  return_decay_0.0 return_decay_0.25  \
accuracy       0.6249 ± 0.0146   0.6236 ± 0.0117   0.6236 ± 0.0120   
pwa            0.6343 ± 0.0139   0.6332 ± 0.0144   0.6331 ± 0.0138   
neg_log_loss  -0.6613 ± 0.0057  -0.6628 ± 0.0057  -0.6623 ± 0.0056   
precision      0.6183 ± 0.0177   0.6184 ± 0.0162   0.6179 ± 0.0169   
recall         0.6025 ± 0.0327   0.5966 ± 0.0310   0.5983 ± 0.0317   
f1             0.6101 ± 0.0247   0.6071 ± 0.0227   0.6077 ± 0.0232   

              return_decay_0.5 return_decay_0.75  
accuracy       0.6224 ± 0.0121   0.6225 ± 0.0152  
pwa            0.6336 ± 0.0137   0.6328 ± 0.0143  
neg_log_loss  -0.6618 ± 0.0055  -0.6621 ± 0.0058  
precision      0.6159 ± 0.0176   0.6156 ± 0.0191  
recall         0.5999 ± 0.0297   0.6007 ± 0.0335  
f1             0.6076 ± 0.0225   0.6079 ± 0.0257  

return model achieved the best neg_log_loss score of -0.6613



In [ ]:
all_cms[scheme] = cms
# pprint(all_cms, sort_dicts=False)
pprint(all_cms[best_model], sort_dicts=False)

KeyError: None

##### Sequential Bootstrap

In [ ]:
# Base estimator for use with sequential bootstrapping
# I chose it beacause the default behaviour of RF is to set max_features='sqrt'
base_rf = clone(best_clf).set_params(bootstrap=False, n_estimators=1, random_state=None, n_jobs=1)

seq_rf = SequentiallyBootstrappedBaggingClassifier(
    samples_info_sets=cont_train.t1,
    price_bars_index=bb_df.index,
    estimator=base_rf,
    n_estimators=20, # set low to save time
    max_features=1,
    max_samples=1,
    bootstrap_features=True,
    oob_score=True,
    n_jobs=N_JOBS,
    random_state=random_state,
    verbose=False,
)

w = weighting_schemes[best_scheme]
cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        seq_rf, X_train, y_train, cv_gen, 
        sample_weight_train=w, 
        sample_weight_score=w,
    )
all_cms[scheme] = cms

scheme = 'seq_bootstrap'
for idx, row in cv_scores_df.iterrows():
    all_cv_scores_df.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"

all_cv_scores_df

SequentiallyBootstrappedBaggingClassifier(estimator=RandomForestClassifier(bootstrap=False,
                                                                           criterion='entropy',
                                                                           max_depth=6,
                                                                           min_weight_fraction_leaf=0.05,
                                                                           n_estimators=1,
                                                                           n_jobs=3),
                                          max_features=1, n_estimators=100,
                                          n_jobs=3, oob_score=True,
                                          price_bars_index=DatetimeIndex(['2018-01-01 23:05:00', '2018-01-01 23:10:00',
               '2018-01-01 23:15:00', '2018-01-01 23:20:00',...
2018-01-03 01:30:00   2018-01-03 01:50:00
2018-01-03 02:40:00   2018-01-03 04:00:00
2018-01-03 05:35:00   2018-01

KeyboardInterrupt: 

In [ ]:
w = weighting_schemes[best_scheme]
rf = best_clf.set_params(oob_score=True).fit(
    X_train, y_train, sample_weight=w,
)

time0 = time.time()
seq_rf.set_params(oob_score=True).fit(
    X_train, y_train, sample_weight=w,
)
time1 = pd.Timedelta(seconds=time.time() - time0).round('1s')
print(f"Sequential Bootstrap done in {time1}")

ensembles = {
    "standard_rf": {"classifier": rf, 
                    "pred": rf.predict(X_test),
                    "prob": rf.predict_proba(X_test),
                    "oob": rf.oob_score_,
                },
    "sequential_rf": {"classifier": seq_rf, 
                      "pred": seq_rf.predict(X_test),
                      "prob": seq_rf.predict_proba(X_test),
                      "oob": seq_rf.oob_score_,
                      },
}

scoring_methods = {
            "accuracy": accuracy_score,
            "pwa": probability_weighted_accuracy,
            "neg_log_loss": log_loss,
            "precision": precision_score,
            "recall": recall_score,
            "f1": f1_score,
        }

all_scores_oos = pd.DataFrame()

for clf in ensembles.keys():
    for method, scoring in scoring_methods.items():
        if scoring in (probability_weighted_accuracy, log_loss):
            y_pred = ensembles[clf]["prob"]
        else:
            y_pred = ensembles[clf]["pred"]
        score = scoring(y_test, y_pred)
        if method == "neg_log_loss":
            score *= -1
        all_scores_oos.loc[method, clf] = score
    
all_scores_oos.round(4)

Sequential Bootstrap done in 0 days 00:12:06


,standard_rf,sequential_rf,sequential_rf_avgu,sequential_rf_unweighted_avgu
accuracy,0.4855,0.5040,0.5013,0.5000
pwa,0.4918,0.5024,0.4899,0.4916
neg_log_loss,-0.7232,-0.7016,-0.7036,-0.7033
precision,0.4888,0.5077,0.5054,0.5038
recall,0.4852,0.4728,0.4315,0.4315
f1,0.4870,0.4896,0.4655,0.4649


In [ ]:
winsound.Beep(1000, 1000)